In [4]:
from PyPDF2 import PdfReader
import spacy
from sentence_transformers import SentenceTransformer
import faiss


/home/ad02/generalai/generateAi/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """Extract text from a PDF file."""
    text = ""
    with open(pdf_path, "rb") as pdf_file:
        reader = PdfReader(pdf_file)
        for page in reader.pages:
            text += page.extract_text()
    return text

In [6]:
text = extract_text_from_pdf("/home/ad02/generalai/generateAi/rag/2023 한국항공대 자율주행 로봇 경진대회 소개.pdf")

In [7]:
text

'2023한국항공대학교 자율주행 로봇경진대회소개\n한국항공대학교\nAI융합대학\n1Creative Engineering \nDesign USAR (Urban Search and Rescue)\n•재해발생후로봇을 사용하여 부분적으로 붕괴된 건물과 같은알수없는환경을\n탐색하고 재해로 인해갇혔거나 의식을 잃은희생자 또는생존자를 찾는것이필요\n2\n•원전이 폭파되어 방사능이 유출된 상황등사람이 접근하지 못하는 환경일 경우\n•재해전체상황을 빠르게 파악할 필요있음-> 로봇을 이용Creative Engineering \nDesign USAR Robots\n3\nNIST ( National Institute of Standards and Technology) UNSW ( University of New South Wales)Creative Engineering \nDesign 본대회에서 사용하는 플랫폼 -Turtlebot4\n4\nRGB -D Camera\n2D LiDAR\n(rplidar )\nMobile Base ( 모터및구동부 포함 )Single Board Computer\n(Raspberry Pi 4B)\nUI보드\n(터틀봇 상태 , 사용자 버튼 , \n디스플레이 제공 )Turtlebot4 Tutorial -https://turtlebot.github.io/turtlebot4 -user -manual/Creative Engineering \nDesign 대회미션\n•본대회의 미션은 2D 라이다와 RGB-D 카메라가 장착된 Turtlebot4 로봇을\n사용하여 USAR 문제의 단순화된 버전을 해결하는 것\n•건물의 벽이나 장애물 -> 미로\n•희생자 또는생존자 -> 고유한 색상의 비콘또는April Tag, QR code\n5\nReference -https://mbanderson.github.io/turtlebot_mission_control/Creative Engineering \nDesign 대회미션\n•ROS 기반의 통합된 소프트웨어 (C++ or Python)

In [23]:
def chunk_text_by_characters(text, chunk_size=1000):
    """
    Splits text into chunks of a specified number of characters.
    """
    nlp_ko = spacy.load("en_core_web_md")
    doc = nlp_ko(text)
    chunks = []
    current_chunk = []
    current_length = 0

    for token in doc:
        token_length = len(token.text_with_ws)
        if current_length + token_length > chunk_size:
            chunks.append("".join(current_chunk).strip())
            current_chunk = []
            current_length = 0
        current_chunk.append(token.text_with_ws)
        current_length += token_length

    if current_chunk:
        chunks.append("".join(current_chunk).strip())

    return chunks # Sample text


In [24]:
import spacy

def chunk_text_with_overlap(text, chunk_size=1000, overlap_size=300):
    """
    Splits text into chunks of a specified number of characters with overlap.
    """
    nlp_ko = spacy.load("en_core_web_md")
    doc = nlp_ko(text)
    chunks = []
    current_chunk = []
    current_length = 0

    for token in doc:
        token_length = len(token.text_with_ws)
        # If the current chunk reaches its size limit, we add it to chunks
        if current_length + token_length > chunk_size:
            # Add the current chunk to the list
            chunks.append("".join(current_chunk).strip())
            # Keep the last `overlap_size` characters for the next chunk
            overlap_chunk = "".join(current_chunk)[-overlap_size:]
            # Reset the current chunk and current length
            current_chunk = [overlap_chunk]
            current_length = len(overlap_chunk)
        # Add the token to the current chunk
        current_chunk.append(token.text_with_ws)
        current_length += token_length

    # Add the final chunk if it exists
    if current_chunk:
        chunks.append("".join(current_chunk).strip())

    return chunks


In [26]:

# Get the chunks
chunks = chunk_text_with_overlap(text,500, 200)

# Display the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:\n{chunk}\n")

Chunk 1:
2023한국항공대학교 자율주행 로봇경진대회소개
한국항공대학교
AI융합대학
1Creative Engineering 
Design USAR (Urban Search and Rescue)
•재해발생후로봇을 사용하여 부분적으로 붕괴된 건물과 같은알수없는환경을
탐색하고 재해로 인해갇혔거나 의식을 잃은희생자 또는생존자를 찾는것이필요
2
•원전이 폭파되어 방사능이 유출된 상황등사람이 접근하지 못하는 환경일 경우
•재해전체상황을 빠르게 파악할 필요있음-> 로봇을 이용Creative Engineering 
Design USAR Robots
3
NIST ( National Institute of Standards and Technology) UNSW ( University of New South Wales)Creative Engineering 
Design 본대회에서 사용하는 플랫폼 -Turtlebot4
4
RGB -D Camera
2D LiDAR
(rplidar )
Mobile Base (

Chunk 2:
3
NIST ( National Institute of Standards and Technology) UNSW ( University of New South Wales)Creative Engineering 
Design 본대회에서 사용하는 플랫폼 -Turtlebot4
4
RGB -D Camera
2D LiDAR
(rplidar )
Mobile Base ( 모터및구동부 포함 )Single Board Computer
(Raspberry Pi 4B)
UI보드
(터틀봇 상태 , 사용자 버튼 , 
디스플레이 제공 )Turtlebot4 Tutorial -https://turtlebot.github.io/turtlebot4 -user -manual/Creative Engineering 
Design 대회미션
•본대회의 미션은 2D 라이다와 RGB-D 카메라가 장착된 Turtlebot4 로봇을
사용하여 USAR 문제의 단순화된 버전을 해결하는 것
•건물의 벽이나 장애물 

In [11]:
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

In [12]:
# # Load Sentence Transformers model
# model = SentenceTransformer('all-MiniLM-L6-v2')

# # Generate embeddings
# embeddings = model.encode(chunks)

# # Output embeddings
# for i, chunk in enumerate(chunks):
#     print(f"Chunk: {chunk}")
#     print(f"Embedding: {embeddings[i][:5]}...")  # Displaying first 5 dimensions for brevity

In [13]:
# Generate embeddings for the corpus
corpus_embeddings = model.encode(chunks)


# Initialize a FAISS index
dimension = corpus_embeddings.shape[1]
print(f"Dimension of the embeddings: {dimension}")
index = faiss.IndexFlatL2(dimension)  # Using L2 distance metric
index.add(corpus_embeddings)  # Add embeddings to the index

Dimension of the embeddings: 768


In [14]:
# User query
# query = "Where is the Eiffel Tower located?"
ko_query = "언제 대회가 시작됩니까?"
query_embedding = model.encode([ko_query])

# Perform similarity search
k = 3  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_embedding, k)

In [15]:
distances, indices

(array([[12.333832, 13.250383, 13.480976]], dtype=float32), array([[2, 1, 0]]))

In [17]:
# Retrieve and display results
retrieved_chunks = [chunks[idx] for idx in indices[0]]
for i, chunk in enumerate(retrieved_chunks):
    print(f"Retrieved Chunk {i+1}: {chunk}")
    print()

Retrieved Chunk 1: ative Engineering 
Design 평가기준및제출자료
•평가의 기준
•미션의 완료시간(40점)
•"생존자 "의위치파악의 정확도 (30점)
•탐색하는 동안구축한 지도와 이동한 경로의 퀄리티 (30점)
•제출해야 하는자료
•작성된 ROS기반소프트웨어를 실행하는 데필요한 모든코드, 실행파일및
기타콘텐츠가 포함된 C++ 또는Python 으로작성된 ROS 패키지
•미션을 수행하는 동안수집된 실시간 ROS bag 파일
•USAR 작업을 어떻게 해결했는지 설명하고 시스템 성능을 분석한 보고서
7Creative Engineering 
Design 참고자료리스트
•Turtlebot4 User manual -https://turtlebot.github.io/turtlebot4 -user-manual/
•Turtlebot4 Basic Setup -https://blu -y.github.io/turtle/guide/basic_setup
•Turtlebot4 Basic Tutorial1 -https://blu -y.github.io/turtle/guide/basic_tutorial_1
•Turtlebot4 Basic Tutorial2 -https://blu -y.github.io/turtle/guide/basic_tutorial_2
•Turtlebot4 Image Processing Tutorial -https://blu -y.github.io/turtle/guide/ip_tutorial
•References
•https://mbanderson.github.io/turtlebot_mission_control/
•https://www.nist.gov/news -events/news/2013/06/new -nist-facility -puts-bomb -and-urban-
search -and-rescue -robots -test
•Videos
•https://youtu.be/UfT0TUR4S9o
•https://youtu.be/5XERzM6Z